##### Copyright &copy; 2020 The Alibaba Authors.

# 载图

GraphScope 以 [属性图](https://github.com/tinkerpop/blueprints/wiki/Property-Graph-Model) 建模图数据。 图上的点/边都带有标签（label），每个标签都可能带有许多属性（property)。

在这个教程中，我们将展示 GraphScope 如何载入一张图，包括

- 如何配置图的数据模型（schema）
- 载图的几种简化形式
- 从多种存储中载图
- 从磁盘中序列化/反序列化图

## 定义图数据模型

首先，创建会话并导入相关的包

In [ ]:
import os
import graphscope
from graphscope.framework.graph import Graph
from graphscope.framework.loader import Loader
import vineyard

k8s_volumes = {
    "data": {
        "type": "hostPath",
        "field": {
          "path": "/testingdata",  # Path in host
          "type": "Directory"
        },
        "mounts": {
          "mountPath": "/home/jovyan/datasets",  # Path in pods
          "readOnly": True
        }
    }
}

graphscope.set_option(show_log=True)  # enable logging
sess = graphscope.session(k8s_volumes=k8s_volumes, k8s_etcd_mem='512Mi')  # create a session

我们使用函数 `load_from` 来载入图。`load_from` 将

1. 配置点和边的相关项如标签名，数据地址，属性列表等
2. 对配置做简单的正确性验证
3. 将数据载入内存，构造一个 `graphscope.Graph` 对象。

`load_from` 的函数签名如下

```python
load_from(edges, vertices=None, directed=True, oid_type="int64_t", generate_eid=True)
```

接下来，我们将逐一介绍这些参数。

### edges

必选参数。

edges 是一个 `Dict`（字典），字典中的每一项键值对都代表一个边类的标签。具体来说，每一项的键是标签的名字， 其值则是一个 `Tuple`（元组） 或 `List`（列表），包含以下几项：

- `Loader` 对象，代表数据源，告知 graphscope 可以在哪里找到源数据，可以是文件路径，或者 numpy 数组等；

- 一组属性名字。属性名应当与数据中的首行表头中的名字相一致。可选项，如果省略或为空，除起始点列和目标点列之外的所有列都将会作为属性载入；

- 定义边的起始点的元组，格式为（起点列名，起点标签名）；

- 定义边的目标点的元组，格式为（目标列名，目标标签名）；


让我们来看一个例子：

In [ ]:
edges={
    # a kind of edge with label "knows"
    "knows": (
        # the data source, in this case, is a file location.
        Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|"),
        # selected column names that would be load as properties
        ["creationDate"],
        # use 'Person.id' column as source id, the src label should be 'person'
        ("Person.id", "person"),
        # use 'Person.id.1' column as destination id, the dst label is 'person'
        ("Person.id.1", "person")
    )
}

注意到这里有个名为 `person` 的域，我们将在下一节详细介绍。

在这里 `Loader` 是一个封装了如何载入数据的类，包含数据路径（包括 本地， HDFS， 亚马逊 S3，或阿里云 OSS），列分隔符以及一些其他元信息。

在这里，`Loader` 指示一个挂载在 Pod 里的文件路径。

### vertices

可选项，默认为 `None`。其值只有在整张图中有且仅有一个点标签，且点上没有任何属性时才可以为 `None`，此时，点 ID 可从所有边的端点中推理得出。

和边类似，一个点的字典（`Dict`） 包含一个键作为标签名，以及一系列配置项作为值。配置项包含如下几项
 
 - Loader object，代表数据源，指示 graphscope 可以在哪里找到源数据，可以为文件路径，或者 numpy 数组等；

 - 一组属性名字。属性名应当与数据中的首行表头中的名字相一致。可选项，如果省略或为空，除ID列之外的所有列都将会作为属性载入；

 - 作为ID列的列名，此列将会载入边时被用来做起始点ID或目标点ID。

这里是一个点配置的例子:

In [ ]:
vertices={
    "person": (
        # source file for vertices labeled as person;
        Loader("/home/jovyan/datasets/ldbc_sample/person_0_0.csv", delimiter="|"),
        # columns loaded as property
        ["firstName", "lastName"],
        # the column used as vertex_id
        "id"
    )
}

### directed

可选项，默认为 `True`.

参数 `directed` 代表以有向还是无向的方式载入图。

In [1]:
directed = True

### oid_type

可选项，默认为 `int64_t`.

参数 `oid_type` 指示图上的 ID 的数据类型。可选的选项为 `string` 或 `int64_t`。我们推荐尽量使用 `int64_t` 因为相比于 `string` 可以节省大量内存，并且在速度上也有提升。

In [2]:
oid_type = 'int64_t'

### generate_eid

可选项，默认为 `True`.

在某些情况下，比如 [Graph Interactive Engine](https://graphscope.io/docs/interactive_engine.html) 需要为每一条边指定一个 ID。

将 `generate_eid` 置为 `True` 将为每条边自动生成一个 ID，简单来说，如果你需要将图用于交互式引擎，设置其为 `True`，否则的话置为 `False`。

在这个教程里我们将其设为 `False`.

In [3]:
generate_eid = False

现在，我们将参数组合在一起来载入图。

In [ ]:
graph = sess.load_from(edges, vertices, directed, oid_type, generate_eid)

现在，我们已经成功的在 GraphScope 中载入了一张图，其包含一种点标签，为 `person`，以及一种边标签，为 `knows`。我们可以查看一下图的配置模型：

In [ ]:
print(graph.schema)

## 序列化与反序列化

当图的规模很大时，可能要花大量时间载入（可能多达几小时）。

GraphScope 提供了序列化与反序列化图数据的功能，可以将载入的图以二进制的形式序列化到磁盘上，以及从这些文件反序列化为一张图。

### 序列化

`graph.serialize` 需要一个 `path` 的参数，代表写入二进制文件的路径。

In [4]:
graph.serialize('/tmp/seri')

NameError: name 'graph' is not defined

### 反序列化

`graph.deserialize` 的参数类似 `graph.serialize`. 但是，其 `path` 参数必须和序列化时为 `graph.serialize` 提供的 `path` 参数完全一致，因为 GraphScope 依赖命名规则去找到所有文件，注意在序列化时，所有的工作者都将其自己所持有的图数据写到一个以自己的工作者ID结尾的文件中，所以在反序列化时的工作者数目也必须和序列化时的工作者数目 **完全一致**。

`graph.deserialize` 额外需要一个 `sess` 的参数，代表将反序列化后的图载入到此会话。

In [ ]:
deserialized_graph = graph.deserialize('/tmp/seri', sess)

## 多种形式载图

回忆一下上一节中定义的边，其使用一个元组去保存多个配置项。

配置项也可以由一个字典来定义，我们有若干保留字，分别为 `loader`, `properties`, `source` 和 `destination`。 下面定义的边和上一节的效果完全一致。

In [5]:
edges={
    "knows": (
        Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|"),
        ["creationDate"],
        ("Person.id", "person"),
        ("Person.id.1", "person")
    )
}

NameError: name 'Loader' is not defined

In [3]:
edges = {
    "knows": {
            "loader": Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|"),
            "properties": ["creationDate"],
            "source": ("Person.id", "person"),
            "destination": ("Person.id.1", "person"),
        },
    }

在一些情况下，一种边标签也许连接着多种点标签。在 LDBC 图中，**likes** 的边标签包含两个关系，比如在论坛中，人们（Person）可以 **like** 帖子(Post)，也可以 **立刻** 帖子下的评论(Comment)，这两种关系可以被表示为 person **likes** post, 以及 person **likes** comments。在这种情况下，标签 **likes** 的值可以是一个列表。

In [ ]:
edges={
    # 边标签为 "likes"
    "likes": [
        (
            Loader("/home/jovyan/datasets/ldbc_sample/person_likes_comment_0_0.csv", delimiter="|"),
            ["creationDate"],
            ("Person.id", "person"),
            ("Comment.id", "comment")
        ),
        (
            Loader("/home/jovyan/datasets/ldbc_sample/person_likes_post_0_0.csv", delimiter="|"),
            ["creationDate"],
            ("Person.id", "person"),
            ("Post.id", "post")
        )
    ]
}

对于边来说，一些域可以被省略。
- 如果 `Loader` 只包含一个 URL，那么可以省略 `Loader` 这个类，只写 URL。
- 属性列表可以为空，代表选择所有列。

In [ ]:
edges={
    "knows": (
        "/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv",
        [],
        ("Person.id", "person"),
        ("Person.id.1", "person")
    )
}

另外，所有的列名都可以用其在数据中的索引值来表示。 比如，在表示源点和终点的元组中的数字表示将第0列作为源点，将第1列作为终点。

In [ ]:
edges={
    "knows": (
        Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|"),
        ["creationDate"],
        # 0 represents the first column.
        (0, "person"),
        # second column used as dst.
        (1, "person"),
    )
}

进一步的，当图中只有一个点的标签时，边的源点和终点的元组也可以被省略，此时 GraphScope 认为所有边的源点和终点都为这个唯一的无歧义的点标签。

In [ ]:
edges={
    "group": (
        Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|"),
        ["creationDate"]
    )
}

最简单的形式可以只包含一个路径，默认情况下，第0列将作为源点 ID，第1列将作为终点 ID，所有其他列都将作为属性载入。

In [ ]:
edges={
    "knows": Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|")
}

Like the edges, the configuration for vertices can also be a Dict, in which the keys are “loader”, “properties” and “vid”

In [ ]:
vertices={
    "person": {
        "loader": Loader("/home/jovyan/datasets/ldbc_sample/person_0_0.csv", delimiter="|"),
        "properties": ["firstName", "lastName"],
        "vid": "id",
    },
}

对于点，同样可以省略某些配置。

- 如果 `Loader` 只包含一个 URL，我们可以省略 `Loader`，只写入 URL。
- 属性列表可以为空，代表载入所有列
- id 可以为数字，表示在源数据中的 index。

在最简单的情况下，点的配置可以只包含一个 URL。此时，第0列将作为 ID， 其他所有列作为属性。

In [ ]:
vertices={
    "person": Loader("/home/jovyan/datasets/ldbc_sample/person_0_0.csv", delimiter="|")
}

最后，我们可以省略掉点的所有配置。 GraphScope 将会从边的端点中推理出点ID，并将 `_` 作为点的标签。

In [ ]:
g = sess.load_from(
    edges={
        "knows": Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|")
        }
    )

我们可以用讲到的知识来载入一张复杂一些的图。

In [ ]:
g = sess.load_from(
    edges={
        "knows": (
            Loader("/home/jovyan/datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|"),
            ["creationDate"],
            ("Person.id", "person"),
            ("Person.id.1", "person")
        ),
        "likes": [
            (
                Loader("/home/jovyan/datasets/ldbc_sample/person_likes_comment_0_0.csv", delimiter="|"),
                ["creationDate"],
                ("Person.id", "person"),
                ("Comment.id", "comment")
            ),
            (
                Loader("/home/jovyan/datasets/ldbc_sample/person_likes_post_0_0.csv", delimiter="|"),
                ["creationDate"],
                ("Person.id", "person"),
                ("Post.id", "post")
            )
        ]
    },
    vertices={
        "person": (
            Loader("/home/jovyan/datasets/ldbc_sample/person_0_0.csv", delimiter="|"),
            ["firstName", "lastName"],
            "id",
        ),
        "comment": (
            Loader("/home/jovyan/datasets/ldbc_sample/comment_0_0.csv", delimiter="|"),
            ["creationDate"],
            "id",
        ),
        "post": (
            Loader("/home/jovyan/datasets/ldbc_sample/post_0_0.csv", delimiter="|"),
            ["creationDate"],
            "id",
        )
    },
)

## 从多种存储中载图

`Loader` 定义了如何载入数据，包含路径，元信息以及其他配置。GraphScope 支持将路径将路径指定为一个 `str`。当接到 `Loader` 的请求时， GraphScope 解析这个字符串并调用 `Vineyard` 中相应协议的 driver。目前，支持的存储包括本地，亚马逊 S3，阿里云OSS， HDFS 以及网络URL。

额外的，GraphScope 也支持从符合特定格式的 `pandas.DataFrame` 或 `numpy.ndarray` 中载图。

载入数据的部分通过 `Vineyard` 来完成，`Vineyard` 利用 `fsspec` 来解析协议，并处理不同的格式。额外的配置参数可以通过关键字参数传递给 `Loader`， 比如为 HDFS 指定 `host` 和 `port`， 为 OSS 和 S3 指定 `access-id` 和 `secret-access-key`.

### Graphs from Location

`Loader` 可以只包含一个字符串。 字符串格式遵循标准的 URI 标准。

In [ ]:
ds1 = Loader("file:///var/datafiles/edgefile.e")

用户需要提供 `key` 和 `secret` 以从 S3 中载图。额外的参数可以通过 `client_kwargs` 来传递, 比如桶的区域 `region_name`.

In [ ]:
ds2 = Loader("s3://bucket/datafiles/edgefile.e", key='access-id', secret='secret-access-key', client_kwargs={'region_name': 'us-east-1'})

用户需要提供 `key`，`secret` 以及桶所在的的 `endpoint` 以从 OSS 中载图。

In [ ]:
ds3 = Loader("oss://bucket/datafiles/edgefile.e", key='access-id', secret='secret-access-key', endpoint='oss-cn-hangzhou.aliyuncs.com')

用户需要提供 `host` 和 `port` 以从 HDFS 中载图，额外的参数可以通过 `extra_conf` 传递。

In [ ]:
ds4 = Loader("hdfs:///var/datafiles/edgefile.e", host='localhost', port='9000', extra_conf={'conf1': 'value1'})

我们来看一个从 S3 中载图的例子。

In [ ]:
graph = sess.load_from(
    edges={
        "knows": (
                Loader("s3://datasets/ldbc_sample/person_knows_person_0_0.csv", delimiter="|", key='testing', secret='testing', client_kwargs={
                    "endpoint_url": "http://192.168.0.222:5000"
                }),
            )
    },
    vertices={
        "person": (
            Loader("s3://datasets/ldbc_sample/person_0_0.csv", delimiter="|", key='testing', secret='testing', client_kwargs={
                    "endpoint_url": "http://192.168.0.222:5000"
                }),
        ),
    },
)
print(graph.schema)

### 从 Numpy 和 Pandas 中载图

对于 `pandas` 来说，其 `DataFrame` 的格式应和 CSV 文件中数据的格式一致。
当前支持的数据类型只包括 `int64` 和 `float64`。

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
leader_id = np.array([0, 0, 0, 1, 1, 3, 3, 6, 6, 6, 7, 7, 8])
member_id = np.array([2, 3, 4, 5, 6, 6, 8, 0, 2, 8, 8, 9, 9])
group_size = np.array([4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2])
e_data = np.transpose(np.vstack([leader_id, member_id, group_size]))
df_group = pd.DataFrame(e_data, columns=['leader_id', 'member_id', 'group_size'])

In [ ]:
student_id = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
avg_score = np.array([490.33, 164.5 , 190.25, 762. , 434.2, 513. , 569. ,  25. , 308. ,  87. ])
v_data = np.transpose(np.vstack([student_id, avg_score]))
df_student = pd.DataFrame(v_data, columns=['student_id', 'avg_score']).astype({'student_id': np.int64})

In [ ]:
# use a dataframe as datasource, properties omitted, col_0/col_1 will be used as src/dst by default.
# (for vertices, col_0 will be used as vertex_id by default)
g = sess.load_from(edges=df_group, vertices=df_student)

对于 `numpy` 来说，可以传入一组COO格式的数组。

In [ ]:
array_group = [df_group[col].values for col in ['leader_id', 'member_id', 'group_size']]
array_student = [df_student[col].values for col in ['student_id', 'avg_score']]

g = sess.load_from(edges=array_group, vertices=array_student)

最后，关闭会话以释放所有资源。

In [ ]:
sess.close()